# Load Bronze Data to Table - ProductCategory

## Overview
Load ProductCategory data from the Bronze CSV file to the Delta table in the lakehouse.

## Data Flow
- **Source**: MAAG_LH_Bronze/Files/samples_fabric/shared/ProductCategory_Samples.csv
- **Target**: shared.ProductCategory (Delta table in lakehouse)
- **Process**: Read CSV, validate schema, load to Delta table

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, sum as spark_sum
import os

# Configuration - Using correct Fabric cross-lakehouse path from Fabric portal
WORKSPACE_NAME = "Fabric_MAAG"
SOURCE_LAKEHOUSE_NAME = "MAAG_LH_Bronze"
SOURCE_PATH = f"abfss://{WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{SOURCE_LAKEHOUSE_NAME}.Lakehouse/Files/samples_fabric/shared/ProductCategory_Samples.csv"

TARGET_SCHEMA = "shared"
TARGET_TABLE = "ProductCategory"
TARGET_FULL_PATH = f"{TARGET_SCHEMA}.{TARGET_TABLE}"

print(f"🔄 Loading ProductCategory data")
print(f"📂 Source: {SOURCE_PATH}")
print(f"🎯 Target: {TARGET_FULL_PATH}")

# Read CSV from Bronze lakehouse
df = spark.read.option("header", "true").option("inferSchema", "true").csv(SOURCE_PATH)

print(f"✅ Data loaded successfully")
print(f"📊 Records: {df.count()}")
print(f"📋 Columns: {df.columns}")

# Display sample data
print(f"\n📖 Sample data:")
df.show(10, truncate=False)

In [ ]:
# Data quality check: Nulls in key columns
nulls = df.select(
    spark_sum(col("CategoryId").isNull().cast("int")).alias("null_CategoryId"),
    spark_sum(col("CategoryName").isNull().cast("int")).alias("null_CategoryName"),
    spark_sum(col("IsActive").isNull().cast("int")).alias("null_IsActive")
).collect()[0]

print(f"🔍 Null check results:")
print(f"  CategoryId nulls:   {nulls['null_CategoryId']}")
print(f"  CategoryName nulls: {nulls['null_CategoryName']}")
print(f"  IsActive nulls:     {nulls['null_IsActive']}")

if nulls['null_CategoryId'] > 0 or nulls['null_CategoryName'] > 0 or nulls['null_IsActive'] > 0:
    print(f"⚠️ Warning: Nulls found in key columns!")
else:
    print(f"✅ No nulls in key columns.")

In [ ]:
# Write DataFrame to Delta table (overwrite mode)
print(f"💾 Writing data to Delta table: {TARGET_FULL_PATH}")

df.write \
  .format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .saveAsTable(TARGET_FULL_PATH)

print(f"✅ Data written to Delta table")

# Verify the load
result_count = spark.sql(f"SELECT COUNT(*) as count FROM {TARGET_FULL_PATH}").collect()[0]["count"]
print(f"📊 Records in Delta table: {result_count}")

print(f"\n📖 Sample from Delta table:")
spark.sql(f"SELECT * FROM {TARGET_FULL_PATH}").show(10, truncate=False)

print(f"🎉 Bronze to Delta table load complete!")